# 動量策略探索

本 Notebook 用於探索和測試動量策略 (Momentum Strategy)

使用 SMA20 / SMA60 交叉進行趨勢追蹤

In [ ]:
# 導入必要的套件
import sys
import os

# 加入專案根目錄到路徑
project_root = os.path.dirname(os.getcwd())
sys.path.append(project_root)

import pandas as pd
import matplotlib.pyplot as plt

from utils.fetch import fetch_stock_data, save_stock_data, load_stock_data
from strategies.momentum import MomentumStrategy
from backtest.backtester import Backtester
from utils.plot import plot_price_with_indicators, plot_backtest_results

# 設定中文顯示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

%matplotlib inline

## 1. 抓取資料

In [ ]:
# 設定股票代碼
symbol = '2330'  # 台積電

# 抓取資料 (使用 yfinance 作為範例，因為不需要 API token)
df = fetch_stock_data(symbol, source='yfinance')

# 顯示前幾筆資料
print(f"資料筆數: {len(df)}")
df.head()

In [ ]:
# 儲存資料
save_stock_data(symbol, df, data_type='raw')

## 2. 計算技術指標

In [ ]:
# 建立動量策略實例
strategy = MomentumStrategy(short_window=20, long_window=60)

# 計算指標
df_with_indicators = strategy.calculate_indicators(df)

# 顯示最後幾筆資料
df_with_indicators[['close', 'SMA20', 'SMA60']].tail(10)

In [ ]:
# 繪製股價與均線
plot_price_with_indicators(
    df_with_indicators[-252:],  # 最近一年
    indicators=['SMA20', 'SMA60'],
    title=f'{symbol} 股價與均線 (最近一年)'
)

## 3. 生成交易訊號

In [ ]:
# 生成交易訊號
df_signals = strategy.generate_signals(df)

# 查看有訊號的時間點
signals = df_signals[df_signals['signal'] != 0][['close', 'SMA20', 'SMA60', 'signal']]
print(f"交易訊號數量: {len(signals)}")
signals.tail(10)

## 4. 回測策略

In [ ]:
# 建立回測引擎
backtester = Backtester(initial_capital=100000, commission=0.001425)

# 執行回測
results = backtester.run(strategy, df)

# 顯示績效摘要
metrics = backtester.summary()

print("\n=== 回測結果 ===")
for key, value in metrics.items():
    if isinstance(value, float):
        print(f"{key}: {value:.2f}")
    else:
        print(f"{key}: {value}")

In [ ]:
# 繪製回測結果
plot_backtest_results(results, title=f'{symbol} 動量策略回測')

## 5. 參數優化

In [ ]:
from ai.optimize_strategy import StrategyOptimizer

# 建立優化器
optimizer = StrategyOptimizer(df, initial_capital=100000)

# 定義參數網格
param_grid = {
    'short_window': [10, 20, 30],
    'long_window': [50, 60, 70]
}

# 執行網格搜尋
optimization_results = optimizer.grid_search(MomentumStrategy, param_grid)

In [ ]:
# 顯示最佳結果
best_result = optimization_results[0]

print("\n=== 最佳參數組合 ===")
print(f"參數: {best_result['parameters']}")
print(f"\n績效指標:")
for key, value in best_result['metrics'].items():
    if isinstance(value, float):
        print(f"  {key}: {value:.2f}")
    else:
        print(f"  {key}: {value}")

In [ ]:
# 生成給 GPT 的優化建議 prompt
prompt = optimizer.generate_gpt_prompt()
print(prompt)